In [ ]:
!pip install -U langchain-community


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.0/290.0 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.9 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [ ]:
!pip install langchain transformers faiss-cpu datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.8 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which 

In [ ]:
!pip uninstall -y torch torchvision torchaudio

Found existing installation: torch 2.4.0+cu121
Uninstalling torch-2.4.0+cu121:
  Successfully uninstalled torch-2.4.0+cu121
Found existing installation: torchvision 0.19.0+cu121
Uninstalling torchvision-0.19.0+cu121:
  Successfully uninstalled torchvision-0.19.0+cu121
Found existing installation: torchaudio 2.4.0+cu121
Uninstalling torchaudio-2.4.0+cu121:
  Successfully uninstalled torchaudio-2.4.0+cu121


In [ ]:
!pip install torch torchvision torchaudio sentence-transformers

In [ ]:
!pip install -U langchain-huggingface


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 15.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import torch
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
from langchain.vectorstores import FAISS
from transformers import BartTokenizer, BartForQuestionAnswering
from langchain_huggingface import HuggingFacePipeline
import faiss
import numpy as np
from transformers import pipeline

# Load the dataset
data_path = '/content/cleaned_bbc_news_articless.csv'
df = pd.read_csv(data_path)

# Determine device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Combine the columns
def combine_columns(row, include_date=True):
    if include_date:
        return f"{row['cleaned_title']} {row['cleaned_text']} Published on: {row['pubDate']}"
    else:
        return f"{row['cleaned_title']} {row['cleaned_text']}"

df['combined_text'] = df.apply(combine_columns, axis=1)
articles = df['combined_text'].tolist()

# SentenceTransformer for embedding
model_name = 'all-MiniLM-L6-v2'
model = SentenceTransformer(model_name, device=device)
embeddings = model.encode(articles, convert_to_tensor=True)  # This will use GPU if available

# FAISS Index creation
embedding_dim = embeddings.shape[1]
faiss_index = faiss.IndexFlatL2(embedding_dim)
faiss_index.add(np.array(embeddings.cpu()))  # Ensure embeddings are on CPU for FAISS

# Initialize the HuggingFace embedding model for LangChain
hf_embeddings = HuggingFaceEmbeddings(model_name=model_name)

# Create Document objects for each article, adding a source identifier
documents = [Document(page_content=article, metadata={'source': f'Article {i}'}) for i, article in enumerate(articles)]

# Create the FAISS vector store from documents
vector_store = FAISS.from_documents(documents, hf_embeddings)

# Save the FAISS index and embeddings
torch.save(embeddings.cpu(), 'bbc_embeddings.pt')  # Ensure embeddings are on CPU for saving
faiss.write_index(faiss_index, 'bbc_faiss_index.faiss')

# Load the fine-tuned BART model for answer generation
model_name = "phiyodr/bart-large-finetuned-squad2"
tokenizer = BartTokenizer.from_pretrained(model_name)
bart_model = BartForQuestionAnswering.from_pretrained(model_name)
bart_model.to(device)  # Move model to the appropriate device

# Define the QA pipeline with BART
hf_pipeline_object = pipeline(
    "question-answering",
    model=bart_model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1,
    max_length=200,  # Adjust this value as needed
    min_length=50,   # Adjust this value as needed
    length_penalty=2.0
)

# Create the HuggingFacePipeline for use in the chain
hf_pipeline = HuggingFacePipeline(pipeline=hf_pipeline_object)  # Pass the pipeline object

# Create the stuff documents chain
stuff_chain = load_qa_with_sources_chain(
    llm=hf_pipeline,
    chain_type="stuff",
)

# Custom function for retrieval
def get_answer(query, stuff_chain, vector_store):
    # Retrieve relevant documents
    docs = vector_store.similarity_search(query)

    # Combine relevant document texts as the context
    context = " ".join([doc.page_content for doc in docs[:3]])  # Limit to 3 documents

    # Prepare the input in a clearer format for BART
    input_text = {
        'question': query,
        'context': context
    }

    # Generate the answer using the fine-tuned BART model
    result = hf_pipeline_object(input_text)
    return result.get('answer', 'No answer found')

# Query example
query = "What is the impact of climate change?"
answer = get_answer(query, stuff_chain, vector_store)
print("Answer:", answer)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

<ipython-input-7-d4787cfa883f>:75: LangChainDeprecationWarning: This function is deprecated. Refer to this guide on retrieval and question answering with sources: https://python.langchain.com/v0.2/docs/how_to/qa_sources/
See also the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/v0.2/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/v0.2/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/v0.2/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/v0.2/docs/versions/migrating_chains/map_rerank_docs_chain

  stuff_chain = load_qa_with_sources_chain(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Answer: harming mental health


In [ ]:
import pandas as pd
import torch
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
from langchain.vectorstores import FAISS
from transformers import BartTokenizer, BartForQuestionAnswering
from langchain_huggingface import HuggingFacePipeline
import faiss
import numpy as np
from transformers import pipeline
from langchain.schema import Document
# Load the dataset
data_path = '/content/cleaned_bbc_news_articless.csv'
df = pd.read_csv(data_path)

# Determine device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Combine the columns
def combine_columns(row, include_date=True):
    if include_date:
        return f"{row['cleaned_title']} {row['cleaned_text']} Published on: {row['pubDate']}"
    else:
        return f"{row['cleaned_title']} {row['cleaned_text']}"

df['combined_text'] = df.apply(combine_columns, axis=1)
articles = df['combined_text'].tolist()

# SentenceTransformer for embedding
model_name = 'all-MiniLM-L6-v2'
model = SentenceTransformer(model_name, device=device)
embeddings = model.encode(articles, convert_to_tensor=True)  # This will use GPU if available

# FAISS Index creation
embedding_dim = embeddings.shape[1]
faiss_index = faiss.IndexFlatL2(embedding_dim)
faiss_index.add(np.array(embeddings.cpu()))  # Ensure embeddings are on CPU for FAISS

# Initialize the HuggingFace embedding model for LangChain
hf_embeddings = HuggingFaceEmbeddings(model_name=model_name)

# Create Document objects for each article, adding a source identifier
documents = [Document(page_content=article, metadata={'source': f'Article {i}'}) for i, article in enumerate(articles)]

# Create the FAISS vector store from documents
vector_store = FAISS.from_documents(documents, hf_embeddings)

# Save the FAISS index and embeddings
torch.save(embeddings.cpu(), 'bbc_embeddings.pt')  # Ensure embeddings are on CPU for saving
faiss.write_index(faiss_index, 'bbc_faiss_index.faiss')

# Load the fine-tuned BART model for answer generation
model_name = "phiyodr/bart-large-finetuned-squad2"
tokenizer = BartTokenizer.from_pretrained(model_name)
bart_model = BartForQuestionAnswering.from_pretrained(model_name)
bart_model.to(device)  # Move model to the appropriate device

# Define the QA pipeline with BART
hf_pipeline_object = pipeline(
    "question-answering",
    model=bart_model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1,
    max_length=200,  # Adjust this value as needed
    min_length=50,   # Adjust this value as needed
    length_penalty=2.0
)

# Create the HuggingFacePipeline for use in the chain
hf_pipeline = HuggingFacePipeline(pipeline=hf_pipeline_object)  # Pass the pipeline object

# Create the stuff documents chain
stuff_chain = load_qa_with_sources_chain(
    llm=hf_pipeline,
    chain_type="stuff",
)

# Custom function for retrieval


def get_answer(query, stuff_chain, vector_store):
    # Retrieve relevant documents
    docs = vector_store.similarity_search(query)

    # Ensure documents are in Document format
    documents = [Document(page_content=doc.page_content, metadata=doc.metadata) for doc in docs[:3]]  # Limit to 3 documents

    # Prepare the input for the stuff_chain in the expected format
    # Each item in the list is a dictionary representing a QA pair
    inputs = [{
        'question': query,
        'context': '\n\n'.join([doc.page_content for doc in documents]) # Combine document contents
    }]

    # Use the stuff_chain to generate an answer
    # Note: We're assuming stuff_chain is a HuggingFacePipeline object here
    result = stuff_chain(inputs)

    # Extract and return the answer
    return result[0]['answer']  # Extract the answer from the first result

# Query example
query = "What is the impact of climate change?"
answer = get_answer(query, stuff_chain, vector_store)
print("Answer:", answer)


ModuleNotFoundError: No module named 'langchain'

In [ ]:
from transformers import pipeline, BartTokenizer, BartForQuestionAnswering

# Load model and tokenizer
model_name = "phiyodr/bart-large-finetuned-squad2"
tokenizer = BartTokenizer.from_pretrained(model_name)
bart_model = BartForQuestionAnswering.from_pretrained(model_name)

# Define the QA pipeline with adjusted parameters
hf_pipeline_object = pipeline(
    "question-answering",
    model=bart_model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1,
    max_length=400,  # Set higher to allow longer answers
    min_length=150,  # Minimum length for more detailed answers
    length_penalty=2.0,  # Encourage longer answers
    num_beams=4,  # Beam search for more diverse answers
    do_sample=True  # Sampling for a varied response
)

def chunk_context(context, chunk_size=512):
    """Split context into chunks if it's too long for the model to handle at once."""
    return [context[i:i + chunk_size] for i in range(0, len(context), chunk_size)]

def test_pipeline(query, context):
    inputs = {
        'question': query,
        'context': context
    }
    result = hf_pipeline_object(inputs)
    return result.get('answer', 'No answer found')

def get_detailed_answer(query, context):
    # Split the context into chunks
    context_chunks = chunk_context(context)

    detailed_answer = ""

    # Run the model on each chunk and append answers
    for chunk in context_chunks:
        answer = test_pipeline(query, chunk)
        detailed_answer += f" {answer}"

    # Optionally extend or elaborate the answer if needed
    return detailed_answer.strip()

# Test example
context = "climate change harming mental health former climate scientist among growing number people overwhelmed ecoanxiety climate change let doom win project tell worrier bbc get exclusive look new project help student deal rising climate anxiety climate change deal ecoanxiety rachel hurst student university east anglia helping others address climate anxiety"
query = "What is the impact of climate change?"
print(get_detailed_answer(query, context))


climate change harming mental health


In [ ]:
from transformers import pipeline, GPT2Tokenizer, GPT2LMHeadModel

# Load a generative model for more elaborate responses
model_name = "gpt2-large"  # Use GPT-2 large, but you can use GPT-3 (via OpenAI API) or GPT-4
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
gpt_model = GPT2LMHeadModel.from_pretrained(model_name)

# Define the pipeline for generation
generative_pipeline = pipeline(
    "text-generation",
    model=gpt_model,
    tokenizer=tokenizer,
    max_length=300,  # Larger max_length for more detailed answers
    min_length=100,  # Ensure the answer isn't too short
    length_penalty=1.2,
    num_beams=4,
    no_repeat_ngram_size=2,  # Prevent repetitive answers
    do_sample=True  # Sampling for creative answers
)

def get_generative_answer(query, context):
    prompt = f"Question: {query}\nContext: {context}\nAnswer:"
    generated_text = generative_pipeline(prompt, max_length=400, min_length=100, num_return_sequences=1)[0]['generated_text']
    return generated_text

# Test example
context = "climate change harming mental health former climate scientist among growing number people overwhelmed ecoanxiety"
query = "What is the impact of climate change?"
print(get_generative_answer(query, context))



tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.25G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question: What is the impact of climate change?
Context: climate change harming mental health former climate scientist among growing number people overwhelmed ecoanxiety
Answer: Climate change is having an impact on our health and wellbeing. It is causing more extreme weather events such as floods, droughts, heatwaves and storms, increasing the risk of heat-related illness and death, and reducing the availability of water and food. The impacts are already being felt in the UK and around the world.
For example, the number of people suffering from anxiety disorders has increased by more than 50% over the past 30 years, while the proportion of children and young people who suffer from depression has also increased. There are also concerns that the impacts will continue to grow as the planet warms and sea levels rise. This is particularly true for those living in low-lying coastal regions, where the effects of sea level rise are likely to be more severe than for other parts of the globe du

In [ ]:
import pandas as pd
import torch
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
from langchain.vectorstores import FAISS
from transformers import pipeline, GPT2Tokenizer, GPT2LMHeadModel
import faiss
import numpy as np

# Load the dataset
data_path = '/content/cleaned_bbc_news_articless.csv'
df = pd.read_csv(data_path)

# Determine device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Combine the columns
def combine_columns(row, include_date=True):
    if include_date:
        return f"{row['cleaned_title']} {row['cleaned_text']} Published on: {row['pubDate']}"
    else:
        return f"{row['cleaned_title']} {row['cleaned_text']}"

df['combined_text'] = df.apply(combine_columns, axis=1)
articles = df['combined_text'].tolist()

# SentenceTransformer for embedding
model_name = 'all-MiniLM-L6-v2'
model = SentenceTransformer(model_name, device=device)
embeddings = model.encode(articles, convert_to_tensor=True)  # This will use GPU if available

# FAISS Index creation
embedding_dim = embeddings.shape[1]
faiss_index = faiss.IndexFlatL2(embedding_dim)
faiss_index.add(np.array(embeddings.cpu()))  # Ensure embeddings are on CPU for FAISS

# Initialize the HuggingFace embedding model for LangChain
hf_embeddings = HuggingFaceEmbeddings(model_name=model_name)

# Create Document objects for each article, adding a source identifier
documents = [Document(page_content=article, metadata={'source': f'Article {i}'}) for i, article in enumerate(articles)]

# Create the FAISS vector store from documents
vector_store = FAISS.from_documents(documents, hf_embeddings)

# Save the FAISS index and embeddings
torch.save(embeddings.cpu(), 'bbc_embeddings.pt')  # Ensure embeddings are on CPU for saving
faiss.write_index(faiss_index, 'bbc_faiss_index.faiss')

# Load GPT-2 model and tokenizer
model_name = "gpt2-large"  # Use GPT-2 large
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
gpt_model = GPT2LMHeadModel.from_pretrained(model_name)
gpt_model.to(device)  # Move the model to the appropriate device

# Define the pipeline for GPT-2 text generation
generative_pipeline = pipeline(
    "text-generation",
    model=gpt_model,
    tokenizer=tokenizer,
    max_length=300,  # Larger max_length for more detailed answers
    min_length=100,  # Ensure the answer isn't too short
    length_penalty=1.2,
    num_beams=4,
    truncation=True,
    no_repeat_ngram_size=2,  # Prevent repetitive answers
    do_sample=True,  # Sampling for creative answers
    device=0 if torch.cuda.is_available() else -1  # Adjust for GPU or CPU
)

# Function to retrieve relevant documents and generate an answer
def get_generative_answer(query, vector_store, num_docs=3):
    # Retrieve the top `num_docs` relevant documents using FAISS
    docs = vector_store.similarity_search(query, k=num_docs)

    # Combine the content of the retrieved documents into a context
    context = " ".join([doc.page_content for doc in docs])

    # Prepare the input prompt for GPT-2
    prompt = f"Question: {query}\nContext: {context}\nAnswer:"

    # Generate the answer using GPT-2
    generated_text = generative_pipeline(prompt, max_length=300, min_length=100, num_return_sequences=1)[0]['generated_text']

    return generated_text

# Example query
# Allow the user to input a query
user_query = input("Please enter your query: ")

# Get the answer based on the user's query
answer = get_generative_answer(user_query, vector_store)

# Print the generated answer
print("Answer:", answer)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Please enter your query: what about ukraine russia war


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Answer: Question: what about ukraine russia war
Context: ukraine war happening russia total fear lithuania russian opposing vladimir putin war say home country like huge prison Published on: 22/09/2022 23:06 ukraine happened day russia invasion vladimir putin pave way foreign fighter join war russia widens attack Published on: 11/03/2022 21:12 watch look back ukraine war two year bbcs joe inwood reflects two year since russia launched full scale invasion ukraine Published on: 24/02/2024 1:41
Answer:
The war in Ukraine started in April 2014, when Russia annexed the Ukrainian peninsula of Crimea. Since then, the conflict has claimed the lives of more than 6,000 people, according to the United Nations.
Russia has been accused of violating international law by sending troops and weapons into eastern Ukraine, where separatists have been fighting Ukrainian forces for nearly two years. The fighting has resulted in the deaths of hundreds of Ukrainian soldiers and civilians, and the displacemen

In [ ]:
import pandas as pd
import torch
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
from langchain.vectorstores import FAISS
from transformers import pipeline, GPT2Tokenizer, GPT2LMHeadModel
import faiss
import numpy as np

# Load the dataset
data_path = '/content/cleaned_bbc_news_articless.csv'
df = pd.read_csv(data_path)

# Determine device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Combine the columns
def combine_columns(row, include_date=True):
    if include_date:
        return f"{row['cleaned_title']} {row['cleaned_text']} Published on: {row['pubDate']}"
    else:
        return f"{row['cleaned_title']} {row['cleaned_text']}"

df['combined_text'] = df.apply(combine_columns, axis=1)
articles = df['combined_text'].tolist()

# SentenceTransformer for embedding
model_name = 'all-MiniLM-L6-v2'
model = SentenceTransformer(model_name, device=device)
embeddings = model.encode(articles, convert_to_tensor=True)  # This will use GPU if available

# FAISS Index creation
embedding_dim = embeddings.shape[1]
faiss_index = faiss.IndexFlatL2(embedding_dim)
faiss_index.add(np.array(embeddings.cpu()))  # Ensure embeddings are on CPU for FAISS

# Initialize the HuggingFace embedding model for LangChain
hf_embeddings = HuggingFaceEmbeddings(model_name=model_name)

# Create Document objects for each article, adding a source identifier
documents = [Document(page_content=article, metadata={'source': f'Article {i}'}) for i, article in enumerate(articles)]

# Create the FAISS vector store from documents
vector_store = FAISS.from_documents(documents, hf_embeddings)

# Save the FAISS index and embeddings
torch.save(embeddings.cpu(), 'bbc_embeddings.pt')  # Ensure embeddings are on CPU for saving
faiss.write_index(faiss_index, 'bbc_faiss_index.faiss')

# Load GPT-2 model and tokenizer
model_name = "gpt2-large"  # Use GPT-2 large
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
gpt_model = GPT2LMHeadModel.from_pretrained(model_name)
gpt_model.to(device)  # Move the model to the appropriate device

# Define the pipeline for GPT-2 text generation
generative_pipeline = pipeline(
    "text-generation",
    model=gpt_model,
    tokenizer=tokenizer,
    max_length=300,  # Larger max_length for more detailed answers
    min_length=100,  # Ensure the answer isn't too short
    length_penalty=1.2,
    num_beams=4,
    truncation=True,
    no_repeat_ngram_size=2,  # Prevent repetitive answers
    do_sample=True,  # Sampling for creative answers
    device=0 if torch.cuda.is_available() else -1  # Adjust for GPU or CPU
)

#  function to retrieve relevant documents and generate an answer with article publish dates
def get_generative_answer(query, vector_store, num_docs=3):
    # Retrieve the top `num_docs` relevant documents using FAISS
    docs = vector_store.similarity_search(query, k=num_docs)

    # Summarize and combine the content of the retrieved documents into a structured context
    context = ""
    for doc in docs:
        article_text = doc.page_content[:300]  # Reduce the length of each document snippet
        publish_date = doc.metadata.get('source')  # Retrieve the 'source' metadata for the publish date
        context += f"Article published on {publish_date}:\n{article_text}\n\n"

    # Prepare the input prompt for GPT-2
    prompt = (
        f"Question: {query}\n"
        f"Context: Please provide a detailed answer based on the following articles. Include references to the publishing dates of the articles mentioned.\n"
        f"{context}\nAnswer:"
    )

    # Generate the answer using GPT-2 with improved generation control
    generated_text = generative_pipeline(
        prompt,
        max_length=600,  # Increase max_length for more detailed responses
        min_length=200,  # Ensure the answer has enough substance
        num_return_sequences=1,
        temperature=0.6,  # Lower temperature for more focused generation
        num_beams=6,  # Increase beams for more thoughtful generation
        no_repeat_ngram_size=2  # Prevent repetition
    )[0]['generated_text']

    return generated_text


# Example query
# Allow the user to input a query
user_query = input("Please enter your query: ")

# Get the answer based on the user's query
answer = get_generative_answer(user_query, vector_store)

# Print the generated answer
print("Answer:", answer)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Please enter your query: tell me about russia ukraine war


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Answer: Question: tell me about russia ukraine war
Context: Please provide a detailed answer based on the following articles. Include references to the publishing dates of the articles mentioned.
Article published on Article 26:
ukraine war kherson mariupol key russian success strategic historical reason offensive southern ukraine vital russia Published on: 04/03/2022 12:54

Article published on Article 302:
ukraine happened day russia invasion vladimir putin pave way foreign fighter join war russia widens attack Published on: 11/03/2022 21:12

Article published on Article 8256:
ukraine war happening russia total fear lithuania russian opposing vladimir putin war say home country like huge prison Published on: 22/09/2022 23:06


Answer: The war in Ukraine began on April 12, 2014, when Russia annexed the Crimean peninsula from Ukraine. The conflict has since claimed the lives of more than 6,000 people and displaced millions of others. Russia's annexation of Crimea has been condemned by 